In [2]:
import numpy as np
import pandas as pd

In [3]:
tags = pd.read_csv('../input/stacksample/Tags.csv')

In [4]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [5]:
temp = tags['Tag'].value_counts()

In [6]:
#pd.set_option("display.max_rows", None)

In [7]:
# printing the counts - 
#temp

In [8]:
temp = pd.DataFrame(temp)
temp = temp.reset_index()
temp.head()

,index,Tag
0,javascript,124155
1,java,115212
2,c#,101186
3,php,98808
4,android,90659


In [9]:
temp.rename(columns = {'index':'Tag' , 'Tag':'Counts'}, inplace = True)
temp.head()

,Tag,Counts
0,javascript,124155
1,java,115212
2,c#,101186
3,php,98808
4,android,90659


In [10]:
tags = pd.merge(tags , temp , on = 'Tag')

In [11]:
tags.head()

,Id,Tag,Counts
0,80,flex,2430
1,10870,flex,2430
2,14350,flex,2430
3,20510,flex,2430
4,20910,flex,2430


## **Choosing the 20 immediate tags which have count >= 4021 - for a decent data size and less imbalance in classes i.e., tags**

In [12]:
temp.loc[temp["Counts"] >= 4021, :].index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
             97,  98,  99, 100, 101, 102, 103, 104, 105, 106],
           dtype='int64', length=107)

In [13]:
# stores the row indices of the chosen tags (i.e., tags with counts >= 4021) from the table - temp
chosen_tags_indices = list(range(106, -1, -1))[:20]
chosen_tags_names = list(temp.loc[chosen_tags_indices, "Tag"].values)

In [14]:
#tags.where(tags['Counts']>=20345 , inplace = True)
tags = tags.loc[tags["Tag"].isin(chosen_tags_names), :]

In [15]:
tags.head()

,Id,Tag,Counts
2430,80,actionscript-3,4039
2431,10870,actionscript-3,4039
2432,14350,actionscript-3,4039
2433,33590,actionscript-3,4039
2434,63390,actionscript-3,4039


In [16]:
#tags.dropna(inplace = True)
#tags.head()

In [17]:
tags = tags.sort_values(by = 'Id')

In [18]:
tags.head()

,Id,Tag,Counts
2430,80,actionscript-3,4039
263927,330,class,4485
325131,930,loops,4121
343735,1760,unit-testing,4437
343736,3150,unit-testing,4437


In [19]:
tags['Id'] = tags['Id'].astype('int32')

In [20]:
tags.shape

(86225, 3)

### **Tags file ready**

## **Importing the Questions file -**

In [21]:
q = pd.read_csv('../input/stacksample/Questions.csv' , encoding='latin-1')

In [22]:
q.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [23]:
df = q.drop(['OwnerUserId' ,  'CreationDate' , 'ClosedDate' , 'Score'] , axis =1)

In [24]:
def comb_tags_func(x):
    list_tags = x.values
    tag_comb = ' '.join(list_tags)
    return tag_comb

In [25]:
mapping_dict = tags.groupby('Id')['Tag'].apply(comb_tags_func)

In [26]:
#tags.groupby('Id')['Tag'].count().sort_values(ascending = False)

In [27]:
#mapping_dict

In [28]:
df['tag'] = df['Id'].map(mapping_dict)

In [29]:
df.head()

,Id,Title,Body,tag
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,actionscript-3
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,NaN
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,NaN
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,NaN


In [30]:
df.dropna(inplace=True)

In [31]:
df['tag'] = df['tag'].apply(lambda x : x.split())

In [32]:
df.head()

,Id,Title,Body,tag
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,[actionscript-3]
5,330,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[class]
10,930,How do I connect to a database and loop over a...,<p>What's the simplest way to connect and quer...,[loops]
20,1760,.NET Unit Testing packages?,<p>Getting back into a bit more .NET after a f...,[unit-testing]
36,3150,How to set up unit testing for Visual Studio C++,<p>I'm having trouble figuring out how to get ...,[unit-testing]


In [33]:
df['Body'].iloc[1]

"<p>I am working on a collection of classes used for video playback and recording. I have one main class which acts like the public interface, with methods like <code>play()</code>, <code>stop()</code>, <code>pause()</code>, <code>record()</code> etc... Then I have workhorse classes which do the video decoding and video encoding. </p>\n\n<p>I just learned about the existence of nested classes in C++, and I'm curious to know what programmers think about using them. I am a little wary and not really sure what the benefits/drawbacks are, but they seem (according to the book I'm reading) to be used in cases such as mine.</p>\n\n<p>The book suggests that in a scenario like mine, a good solution would be to nest the workhorse classes inside the interface class, so there are no separate files for classes the client is not meant to use, and to avoid any possible naming conflicts? I don't know about these justifications. Nested classes are a new concept to me. Just want to see what programmers 

## **Text cleaning and preprocessing -**

In [34]:
df['Body']=df['Body'].str.replace('<p>',' ')

In [35]:
import re
df['Body']=df['Body'].apply(lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x))

In [36]:
df['Body']=df['Body'].str.replace('</p>',' ')
df['Body']=df['Body'].str.replace('\n',' ')
df['Body']=df['Body'].str.replace('</a>',' ')

In [37]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(nltk.corpus.stopwords.words('english'))
df['Body']=df['Body'].apply(lambda x : x.lower())
df['Body'] = df['Body'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
df['Body']=df['Body'].str.replace('<a href=" ">','')

In [39]:
df['Body'] = df['Body'].apply(lambda x : ' '.join([w for w in x.split() if len(w)>3]))

In [40]:
df['Body'].iloc[1]

'working collection classes used video playback recording. main class acts like public interface, methods like <code>play()</code>, <code>stop()</code>, <code>pause()</code>, <code>record()</code> learned existence nested classes c++, curious know programmers think using them. little wary really sure benefits/drawbacks are, seem (according book reading) used cases mine. book suggests scenario like mine, good solution would nest workhorse classes inside interface class, separate files classes client meant use, avoid possible naming conflicts? know justifications. nested classes concept want programmers think issue.'

In [43]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

stemmer = SnowballStemmer('english')
stop_words=set(nltk.corpus.stopwords.words('english'))

df['Title'] = [i.replace("?", "") for i in df['Title']]
df['Title'] = df['Title'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))
df['Title'] = [stemmer.stem(i) for i in df['Title']]

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Model building begins -**

In [44]:
df.head()

,Id,Title,Body,tag
0,80,sqlstatement.execute() - multiple queries one ...,i've written database generation script want e...,[actionscript-3]
5,330,should i use nested classes cas,working collection classes used video playback...,[class]
10,930,how i connect database loop recordset c#,what's simplest connect query database records,[loops]
20,1760,.net unit testing packag,"getting back .net few-years using full-time, w...",[unit-testing]
36,3150,how set unit testing visual studio c++,trouble figuring testing framework usable visu...,[unit-testing]


In [45]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

s=df['tag']
tag_dummies=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index).add_prefix('tag_')
df=pd.concat([df,tag_dummies],axis=1)

In [46]:
df.head()

,Id,Title,Body,tag,tag_actionscript-3,tag_android-layout,tag_class,tag_date,tag_email,tag_google-chrome,...,tag_python-3.x,tag_sockets,tag_sorting,tag_symfony2,tag_tsql,tag_unit-testing,tag_validation,tag_visual-studio-2010,tag_wcf,tag_xaml
0,80,sqlstatement.execute() - multiple queries one ...,i've written database generation script want e...,[actionscript-3],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,330,should i use nested classes cas,working collection classes used video playback...,[class],0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,930,how i connect database loop recordset c#,what's simplest connect query database records,[loops],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,1760,.net unit testing packag,"getting back .net few-years using full-time, w...",[unit-testing],0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
36,3150,how set unit testing visual studio c++,trouble figuring testing framework usable visu...,[unit-testing],0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [47]:
pred_features=[feature for feature in df.columns if 'tag_' in feature]
pred_features

['tag_actionscript-3',
 'tag_android-layout',
 'tag_class',
 'tag_date',
 'tag_email',
 'tag_google-chrome',
 'tag_google-maps',
 'tag_jsp',
 'tag_loops',
 'tag_maven',
 'tag_python-3.x',
 'tag_sockets',
 'tag_sorting',
 'tag_symfony2',
 'tag_tsql',
 'tag_unit-testing',
 'tag_validation',
 'tag_visual-studio-2010',
 'tag_wcf',
 'tag_xaml']

In [48]:
y = df[pred_features]

In [49]:
y.shape

(84884, 20)

In [50]:
df['text'] = df['Body'] + ' ' + df['Title']
df['num_tags'] = df.loc[:, "tag_actionscript-3":"tag_xaml"].sum(axis=1)
df.head()
#X = df['text']

,Id,Title,Body,tag,tag_actionscript-3,tag_android-layout,tag_class,tag_date,tag_email,tag_google-chrome,...,tag_sorting,tag_symfony2,tag_tsql,tag_unit-testing,tag_validation,tag_visual-studio-2010,tag_wcf,tag_xaml,text,num_tags
0,80,sqlstatement.execute() - multiple queries one ...,i've written database generation script want e...,[actionscript-3],1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,i've written database generation script want e...,1
5,330,should i use nested classes cas,working collection classes used video playback...,[class],0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,working collection classes used video playback...,1
10,930,how i connect database loop recordset c#,what's simplest connect query database records,[loops],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,what's simplest connect query database records...,1
20,1760,.net unit testing packag,"getting back .net few-years using full-time, w...",[unit-testing],0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"getting back .net few-years using full-time, w...",1
36,3150,how set unit testing visual studio c++,trouble figuring testing framework usable visu...,[unit-testing],0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,trouble figuring testing framework usable visu...,1


In [51]:
df['num_tags'].value_counts()

1    83554
2     1319
3       11
Name: num_tags, dtype: int64

### **Maintaining the ratio of the number of tags(for each question) in each fold -**

In [52]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense , Dropout , LSTM , Bidirectional , Embedding



In [53]:
df['kfold'] = -1
df = df.sample(frac = 1, random_state = 100).reset_index(drop = True)

In [54]:
skf = model_selection.StratifiedKFold(n_splits = 5)

# Filling the new KFold column - 
for f , (t_ , v_) in enumerate(skf.split(X = df , y = df.num_tags.values)) :
    df.loc[v_ , 'kfold'] = f

In [55]:
df.kfold.value_counts()

0    16977
1    16977
2    16977
3    16977
4    16976
Name: kfold, dtype: int64

In [56]:
lr=LogisticRegression(penalty='l1',solver='liblinear',class_weight='balanced',C=0.1, random_state=25)

In [57]:
""""import keras
inputs = keras.Input(shape=(2500,))
x = keras.layers.Dense(128, activation='relu')(inputs)
x = keras.layers.Dense(64, activation = 'relu')(x)
x = keras.layers.Dense(32, activation = 'relu')(x)
x = keras.layers.Dense(16, activation = 'relu')(x)
output1 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output2 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output3 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output4 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output5 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output6 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output7 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output8 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output9 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output10 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output11 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output12 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output13 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output14 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output15 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output16 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output17 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output18 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output19 = keras.layers.Dense(1, activation = 'sigmoid')(x)
output20 = keras.layers.Dense(1, activation = 'sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=[output1 , output2, output3, output4, output5, output6, output7, output8, output9, output10, output11, output12, output13, output14, output15, output16, output17, output18, output19, output20])
model.compile(optimizer="rmsprop",
              loss=["binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy"],
              metrics=[["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"],["accuracy"]])
              """

'"import keras\ninputs = keras.Input(shape=(2500,))\nx = keras.layers.Dense(128, activation=\'relu\')(inputs)\nx = keras.layers.Dense(64, activation = \'relu\')(x)\nx = keras.layers.Dense(32, activation = \'relu\')(x)\nx = keras.layers.Dense(16, activation = \'relu\')(x)\noutput1 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput2 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput3 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput4 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput5 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput6 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput7 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput8 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput9 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput10 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput11 = keras.layers.Dense(1, activation = \'sigmoid\')(x)\noutput12 = keras.layers.Dense(1, activation

In [58]:
import keras
inputs = keras.Input(shape=(2500,))
x = keras.layers.Dense(128, activation='relu')(inputs)
x = keras.layers.Dense(64, activation = 'relu')(x)
x = keras.layers.Dense(32, activation = 'relu')(x)
x = keras.layers.Dense(16, activation = 'relu')(x)
output = keras.layers.Dense(20, activation = 'sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=output)

2022-03-30 17:22:49.178149: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 17:22:49.266817: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 17:22:49.267638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 17:22:49.269073: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [59]:
model.compile(optimizer="rmsprop",loss="binary_crossentropy" , metrics = ['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2500)]            0         
_________________________________________________________________
dense (Dense)                (None, 128)               320128    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 20)                340       
Total params: 331,332
Trainable params: 331,332
Non-trainable params: 0
_______________________________________________________

In [60]:
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english' , max_features = 2500)

In [61]:
X = df['text']
y = df[pred_features]

In [62]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2)

In [63]:
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [64]:
y_train.shape

(67907, 20)

In [65]:
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
X_test = pd.DataFrame.sparse.from_spmatrix(X_test)

In [66]:
model.fit(X_train , y_train , epochs = 5)

2022-03-30 17:23:07.903590: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1358140000 exceeds 10% of free system memory.
2022-03-30 17:23:09.334792: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1358140000 exceeds 10% of free system memory.
2022-03-30 17:23:10.567652: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
2123/2123 [==============================] - 9s 4ms/step - loss: 0.0824 - accuracy: 0.7411
Epoch 2/5
2123/2123 [==============================] - 7s 3ms/step - loss: 0.0431 - accuracy: 0.8706
Epoch 3/5
2123/2123 [==============================] - 7s 3ms/step - loss: 0.0381 - accuracy: 0.8855
Epoch 4/5
2123/2123 [==============================] - 7s 3ms/step - loss: 0.0349 - accuracy: 0.8947
Epoch 5/5
2123/2123 [==============================] - 7s 3ms/step - loss: 0.0323 - accuracy: 0.9017


In [67]:
model.evaluate(X_test , y_test)

531/531 [==============================] - 1s 2ms/step - loss: 0.0398 - accuracy: 0.8824


[0.039807550609111786, 0.8823702931404114]

In [68]:
#inp = np.array(X_test.iloc[0])
inp = X_test.iloc[0]
inp = inp.tolist()
#inp = pd.DataFrame.sparse.from_spmatrix(inp)
inp = np.reshape(inp , (-1 , 1))

In [69]:
inp = np.transpose(inp)

In [70]:
model.predict(inp)

array([[1.28910478e-05, 1.65781422e-09, 1.17473639e-04, 1.57322245e-06,
        1.57761718e-07, 2.99247354e-03, 7.22193436e-05, 1.18961747e-04,
        6.91451060e-05, 3.94267986e-11, 6.12116361e-04, 1.18628705e-04,
        9.99385595e-01, 2.20100549e-10, 1.49306325e-05, 9.12926845e-10,
        1.01451882e-10, 1.46928374e-04, 1.41304980e-12, 8.49972897e-14]],
      dtype=float32)

In [71]:
std_obj = StandardScaler()

In [72]:
#sgd_model = SGDClassifier(loss='hinge', alpha=0.00001, penalty='elasticnet', l1_ratio = 0.70, max_iter = 1000, tol = 1e-3, learning_rate = 'adaptive', 
                          #early_stopping = True, n_iter_no_change = 3, class_weight = None, eta0 = 0.1)
#multilabel_model = MultiOutputClassifier(sgd_model, n_jobs=-1)

#multilabel_model = MultiOutputClassifier(model1, n_jobs = -1)
#multilabel_model = BinaryRelevance(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'))
#multilabel_model = ClassifierChain(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'))
#multilabel_model = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
#multilabel_model = MultiOutputClassifier(DecisionTreeClassifier(max_depth = 11, class_weight = 'balanced'), n_jobs=-1)
#multilabel_model = MultiOutputClassifier(lr, n_jobs = -1)

In [73]:
multilabel_model = MultiOutputClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs = -1)

In [74]:
fold_num = 1
#log_reg_coefs = []
#log_reg_intercepts = []
train_acc_scores = []
val_acc_scores = []
train_ham_loss = []
val_ham_loss = []
for kfold_idx in df['kfold'].value_counts().index:
    print(f"Fold num : {fold_num}", end = "\n")
    X_train = df[df['kfold'] != kfold_idx].loc[:, 'text'].values
    X_train = tfidf_vectorizer.fit_transform(X_train)
   #X_train = std_obj.fit_transform(X_train)
    y_train = df[df['kfold'] != kfold_idx].loc[:, "tag_actionscript-3":"tag_xaml"].values
    X_val = df[df['kfold'] == kfold_idx].loc[:, 'text'].values
    X_val = tfidf_vectorizer.transform(X_val)
    #X_val = std_obj.fit_transform(X_val)
    y_val = df[df['kfold'] == kfold_idx].loc[:, "tag_actionscript-3":"tag_xaml"].values
    
    
    multilabel_model.fit(X_train , y_train)
    #lr.fit(X_train, y_train)
    #log_reg_coefs.append(list(lr.coef_))
    #log_reg_intercepts.append(lr.intercept_)
    preds_train = multilabel_model.predict(X_train)
    preds_val = multilabel_model.predict(X_val)
    
    train_acc_scores.append(accuracy_score(y_true = y_train, y_pred = preds_train))
    val_acc_scores.append(accuracy_score(y_true = y_val, y_pred = preds_val))
    train_ham_loss.append(hamming_loss(y_true = y_train, y_pred = preds_train))
    val_ham_loss.append(hamming_loss(y_true = y_val, y_pred = preds_val))
    
    print(f"Train Accuracy : {train_acc_scores[-1]}")
    print(f"Val Accuracy : {val_acc_scores[-1]}")
    
    print(f"Train Hamming Loss : {train_ham_loss[-1]}")
    print(f"Val Hamming Loss : {val_ham_loss[-1]}")
    
    print("\n")
    fold_num = fold_num + 1
print(f"Mean Train Accuracy : {np.mean(train_acc_scores)}")
print(f"Mean Val Accuracy : {np.mean(val_acc_scores)}")

print(f"Mean Train Hamming Loss : {np.mean(train_ham_loss)}")
print(f"Mean Val Hamming Loss : {np.mean(val_ham_loss)}")

Fold num : 1
Train Accuracy : 0.8021706156949946
Val Accuracy : 0.7800553690286859
Train Hamming Loss : 0.011419294034488344
Val Hamming Loss : 0.012843847558461447


Fold num : 2
Train Accuracy : 0.8031130811256572
Val Accuracy : 0.7752253048241738
Train Hamming Loss : 0.01134934542830636
Val Hamming Loss : 0.01326500559580609


Fold num : 3
Train Accuracy : 0.804968559942274
Val Accuracy : 0.7764033692642988
Train Hamming Loss : 0.011322102286951272
Val Hamming Loss : 0.013105966896389232


Fold num : 4
Train Accuracy : 0.8026860264773882
Val Accuracy : 0.7766978853743299
Train Hamming Loss : 0.01142297554007687
Val Hamming Loss : 0.013164870118395476


Fold num : 5
Train Accuracy : 0.800509512870354
Val Accuracy : 0.7808671065032988
Train Hamming Loss : 0.011506744418919714
Val Hamming Loss : 0.01275624410933082


Mean Train Accuracy : 0.8026895592221337
Mean Val Accuracy : 0.7778498069989574
Mean Train Hamming Loss : 0.011404092341748514
Mean Val Hamming Loss : 0.013027186855676613

### **OneVsRest and MultiOutput almost give the same results**

## **Training and evaluating the best model -**

In [75]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, 'text'], df.loc[:, "tag_actionscript-3":"tag_xaml"], test_size = 0.2, random_state = 25)

In [76]:
# Transforming the X variables before modelling - 
X_train = tfidf_vectorizer.fit_transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

In [77]:
multilabel_model = MultiOutputClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
multilabel_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=1e-05, loss='log',
                                              penalty='l1'),
                      n_jobs=-1)

In [78]:
print("Train acc. score:", accuracy_score(y_true = y_train, y_pred = multilabel_model.predict(X_train)), end = "\n")
print("Train hamming loss:", hamming_loss(y_true = y_train, y_pred = multilabel_model.predict(X_train)), end = "\n\n")
print("Test acc. score:", accuracy_score(y_true = y_test, y_pred = multilabel_model.predict(X_test)), end = "\n")
print("Test hamming loss:", hamming_loss(y_true = y_test, y_pred = multilabel_model.predict(X_test)), end = "\n\n")

Train acc. score: 0.8028627387456374
Train hamming loss: 0.011423711841194575

Test acc. score: 0.7806444012487483
Test hamming loss: 0.012843847558461447



## **Creating a function for the model -**

In [79]:
def model_func(question):
    # processing the question - 
    question = question.replace('<p>',' ')
    
    f1 = lambda x : re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)' , ' ' , x)
    question = f1(question)
    
    question = question.replace('</p>',' ')
    question = question.replace('\n',' ')
    question = question.replace('</a>',' ')
    
    f2 = lambda x : x.lower()
    question = f2(question)
    
    f3 = lambda x : ' '.join([w for w in x.split() if not w in stop_words])
    question = f3(question)
    
    question = question.replace('<a href=" ">','')
    
    f4 = lambda x : ' '.join([w for w in x.split() if len(w)>3])
    question = f4(question)
    
    # applying tfidf - 
    tr_question = tfidf_vectorizer.transform(pd.Series([question]))
    
    # predicting the tags for the input question - 
    return multilabel_model.predict(tr_question)

In [80]:
sample_input = "I'm not able to load tsql in visual-studio and also in chrome , I'm such a noob :("
model_func(sample_input)

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [81]:
sample_output = model_func(sample_input)

In [82]:
# displaying the tags for the sample input - 
mlb.inverse_transform(sample_output)

[('google-chrome', 'tsql')]

In [83]:
import pickle
pickle.dump(multilabel_model, open("sof_first_model.sav", 'wb'))

In [84]:
pickle.dump(tfidf_vectorizer, open("tfidf_object.pickle", "wb"))

In [85]:
pickle.dump(mlb, open("multilabel_binarizer.pickle", "wb"))